![Red10](http://i.imgur.com/pI7TE3w.png)

## Setup the Environment

Load the shared core, methods, and environment before starting processing

In [1]:
from __future__ import print_function
import sys, os, requests, json, datetime

# Load the environment and login the user
from src.common.load_redten_ipython_env import user_token, user_login, csv_file, run_job, core, api_urls, ppj, rt_url, rt_user, rt_pass, rt_email, lg, good, boom, anmt, mark, ppj, uni_key, rest_login_as_user, rest_full_login, wait_for_job_to_finish, wait_on_job, get_job_analysis, get_job_results, get_analysis_manifest, get_job_cache_manifest, build_prediction_results, build_forecast_results, get_job_cache_manifest, search_ml_jobs, show_logs, show_errors, ipyImage, ipyHTML, ipyDisplay, pd, np

anmt("Environment ready")

Environment ready


## Analyze the IRIS dataset with an XGB Regressor

In [2]:
job_id = None # on success, this will store the actively running job's id

resource_url = rt_url + "/ml/run/"
query_params = {}
uni = uni_key()

lg("Building Post Data", 6)

title = "Hello World - " + str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
desc = "This is a Description for - " + str(title)

# initial stubs
feature_column_names = []
target_column_values = []
ignore_features = []
csv_file = "/opt/work/data/src/iris.csv"
rloc = ""
sloc = ""
units_ahead = 0

ds_name = "iris_regressor"
algo_name = "xgb-regressor"

# What column has the labeled targets as integers? (added-manually to the dataset)
target_column_name = "ResultLabel"                 
# possible values in the Target Column
target_column_values = [ "Iris-setosa", "Iris-versicolor", "Iris-virginica" ] 

# What columns can the algorithms use for training and learning?
feature_column_names = [ "SepalLength", "SepalWidth", "PetalLength", "PetalWidth", "ResultTargetValue" ] 

# What column holds string labels for the Target Column?
label_column_name = "ResultLabel"
ignore_features = [ # Prune non-int/float columns as needed: 
                    target_column_name
                ]
if target_column_name != label_column_name:
    ignore_features.append(label_column_name)

predict_row = {
                "SepalLength"       : 5.4,
                "SepalWidth"        : 3.4,
                "PetalLength"       : 1.7,
                "PetalWidth"        : 0.2,
                "ResultTargetValue" : 0
            }

post_data = {
            "title" : title,
            "desc" : desc,
            "predict_this_data" : predict_row,
            "ds_name" : ds_name,
            "feature_column_names" : feature_column_names,
            "ignore_features" : ignore_features,
            "csv_file" : csv_file,
            "rloc" : rloc,
            "sloc" : sloc,
            "algo_name" : algo_name,
            "target_column_values" : target_column_values,
            "target_column_name" : target_column_name,
            "label_column_name" : label_column_name,
            "prediction_type" : "Predict",
            "ml_type" : "Predict with Filter",
            "train" : {
                    "base_score" : 0.5,
                    "col_sample_by_level" : 1.0,
                    "col_sample_by_tree" : 0.8,
                    "gamma" : 0,
                    "learning_rate" : 0.1,
                    "max_delta_step" : 0,
                    "max_depth" : 6,
                    "min_child_weight" : 1,
                    "num_estimators" : 1000,
                    "objective" : "reg:linear",
                    "reg_alpha" : 0.0,
                    "reg_lambda" : 1.0,
                    "scaled_position_weight" : 1.0,
                    "seed" : 27,
                    "sub_sample" : 0.8,
                    "debug" : True
            },
            "user_id" : 2
        }
post_headers = {
                "Content-type" : "application/json",
                "Authorization" : "JWT " + str(user_token)
            }

lg("Running ML Job url=" + str(resource_url), 6)
post_response = requests.post(resource_url, params=query_params, data=json.dumps(post_data), headers=post_headers)

if post_response.status_code != 201 and post_response.status_code != 200:
    lg("Failed with Post Response Status=" + str(post_response.status_code) + " Reason=" + str(post_response.reason), 0)
    lg("Details:\n" + str(post_response.text) + "\n", 0)
else:
    lg("SUCCESS - Post Response Status=" + str(post_response.status_code) + " Reason=" + str(post_response.reason), 5)
    record = post_response.json()
    lg(ppj(record))
    job_id = int(record["id"])
# end of post for running an ML Job

Building Post Data
Running ML Job url=https://redten.io/ml/run/
SUCCESS - Post Response Status=201 Reason=Created
{
    "algo_name": "xgb-regressor",
    "control_state": "active",
    "created": "2017-05-26 17-49-23",
    "csv_file": "/opt/work/data/src/iris.csv",
    "desc": "This is a Description for - Hello World - 2017-05-26-17-49-22",
    "ds_name": "iris_regressor",
    "feature_column_names": [
        "SepalLength",
        "SepalWidth",
        "PetalLength",
        "PetalWidth",
        "ResultTargetValue"
    ],
    "id": 560,
    "ignore_features": [
        "ResultLabel"
    ],
    "manifest_secret_key": null,
    "manifest_sloc": null,
    "max_features": 10,
    "ml_type": "Predict with Filter",
    "prediction_type": "predict",
    "rloc": "",
    "sloc": "",
    "status": "requested",
    "target_column_name": "ResultLabel",
    "target_column_values": [
        "Iris-setosa",
        "Iris-versicolor",
        "Iris-virginica"
    ],
    "title": "Hello World - 2017

## Wait for the job to finish

In [3]:
job_data = {}
job_report = {}

job_res = {}
if job_id == None:
    boom("Failed to start a new job")
else:
    job_res = wait_on_job(job_id)

    if job_res["status"] != "SUCCESS":
        boom("Job=" + str(job_id) + " failed with status=" + str(job_res["status"]) + " err=" + str(job_res["error"]))
    else:
        job_data = job_res["record"]
        anmt("Job Report:")
        lg(ppj(job_data), 5)
# end of waiting

Waiting on job=560 url=https://redten.io/ml/560/
Job=560 is training - Step 3/10
Job=560 is plotting - Step 7/10
Job=560 is uploading - Step 9/10
Job=560 completed
Job Report:
{
    "job": {
        "algo_name": "xgb-regressor",
        "control_state": "active",
        "created": "2017-05-26 17-49-23",
        "csv_file": "/opt/work/data/src/iris.csv",
        "desc": "This is a Description for - Hello World - 2017-05-26-17-49-22",
        "ds_name": "iris_regressor",
        "feature_column_names": [
            "SepalLength",
            "SepalWidth",
            "PetalLength",
            "PetalWidth",
            "ResultTargetValue"
        ],
        "id": 560,
        "ignore_features": [
            "ResultLabel"
        ],
        "images": [
            {
                "author_name": null,
                "desc": null,
                "id": 12158,
                "image": "https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12158_68fe500cd7564213.png",
        

## Get Predictive Accuracies

In [4]:
job_report = {}
if job_id == None:
    boom("Failed to start a new job")
else:
    # Get the analysis, but do not auto-show the plots
    job_report = get_job_analysis(job_id, show_plots=False)
    if len(job_report) == 0:
        boom("Job=" + str(job_id) + " failed")
    else:
        lg("")
    # if the job failed
# end of get job analysis

# Build the predictive accuracy dictionary from the analysis
# and show the prediction dataframes
acc_results = build_prediction_results(job_report)

# for all columns in the accuracy dictionary:
for col in acc_results:
    col_node = acc_results[col]
    
    # Sensitivity vs Specificity
    # https://en.wikipedia.org/wiki/Sensitivity_and_specificity
    # 
    # successful predictions above 90%...how's that error rate though?
    if col_node["accuracy"] > 0.90: 
        good("Column=" + str(col) + " accuracy=" + str(col_node["accuracy"]) + " mse=" + str(col_node["mse"]) + " num_predictions=" + str(len(col_node["predictions_df"].index)))
    # successful predictions between 90% and 80%...how's that error rate though?
    elif 0.90 > col_node["accuracy"] > 0.80:
        lg("Column=" + str(col) + " accuracy=" + str(col_node["accuracy"]) + " mse=" + str(col_node["mse"]) + " num_predictions=" + str(len(col_node["predictions_df"].index)))
    else:
        boom("Column=" + str(col) + " is not very accurate: accuracy=" + str(col_node["accuracy"]) + " mse=" + str(col_node["mse"]) + " num_predictions=" + str(len(col_node["predictions_df"].index)))
    # end of header line

    # show the predictions
    ipyDisplay(col_node["predictions_df"])
    
    lg("")
# end of showing prediction results

Getting analysis for job=560 url=https://redten.io/ml/analysis/560/
SUCCESS - GET Analysis Response Status=200 Reason=OK
Found Job=560 analysis

Column=SepalLength is not very accurate: accuracy=0.783988472083 mse=0.145626284054 num_predictions=66


,0
0,6.200572
1,5.619819
2,7.236766
3,5.094768
4,6.138592
5,5.744506
6,7.033538
7,6.200797
8,5.400625
9,5.872986



Column=PetalWidth accuracy=0.928719140212 mse=0.0427658976593 num_predictions=66


,0
0,1.331062
1,0.345133
2,1.719044
3,1.145035
4,1.600555
5,0.220241
6,1.483402
7,2.085606
8,1.308045
9,1.417230



Column=PetalLength accuracy=0.982132813713 mse=0.0572320923595 num_predictions=66


,0
0,4.513339
1,1.782143
2,6.363951
3,3.937796
4,4.900630
5,1.486664
6,4.613516
7,5.629632
8,3.513844
9,4.433239



Column=SepalWidth is not very accurate: accuracy=0.368379570605 mse=0.114673327959 num_predictions=66


,0
0,3.021737
1,3.531068
2,3.532478
3,2.388528
4,2.615827
5,3.540309
6,2.931330
7,3.020647
8,2.673050
9,2.764662



Column=ResultTargetValue accuracy=0.920473871675 mse=0.0542588735957 num_predictions=66


,0
0,1.020111
1,0.000028
2,2.032431
3,0.996819
4,2.006838
5,0.000028
6,1.000703
7,1.999662
8,0.998231
9,1.712724


## Get the Analysis Images

In [5]:
# Show the images built during the analysis
if "images" in job_report:
    for img in job_report["images"]:
        anmt(img["title"])
        lg("URL: " + str(img["image"]))
        ipyDisplay(ipyImage(url=img["image"]))
        lg("---------------------------------------------------------------------------------------")
else:
    boom("Job=" + str(job_id) + " does not have any images yet. Please wait a few seconds and retry.")
# end of if images exist    

Joint Plot - ResultTargetValue vs SepalLength - /opt/work/data/src/jointplot_IRIS_REGRESSOR-2-560_4
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12158_68fe500cd7564213.png


---------------------------------------------------------------------------------------
Joint Plot - PetalWidth vs SepalLength - /opt/work/data/src/jointplot_IRIS_REGRESSOR-2-560_3
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12157_0180f59977c44413.png


---------------------------------------------------------------------------------------
Joint Plot - PetalLength vs SepalLength - /opt/work/data/src/jointplot_IRIS_REGRESSOR-2-560_2
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12156_4a32649040c14ab6.png


---------------------------------------------------------------------------------------
Joint Plot - SepalWidth vs SepalLength - /opt/work/data/src/jointplot_IRIS_REGRESSOR-2-560_1
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12155_e1d895dcd1714932.png


---------------------------------------------------------------------------------------
Joint Plot - SepalLength vs SepalWidth - /opt/work/data/src/jointplot_IRIS_REGRESSOR-2-560_0
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12154_9b0be64a57fd484b.png


...

---------------------------------------------------------------------------------------
Scatter - SepalWidth vs SepalLength - /opt/work/data/src/scatter_IRIS_REGRESSOR-2-560_1
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12150_8166153a2f87499d.png


---------------------------------------------------------------------------------------
Scatter - SepalLength vs SepalWidth - /opt/work/data/src/scatter_IRIS_REGRESSOR-2-560_0
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12149_bd798d8ff84f46f6.png


---------------------------------------------------------------------------------------
Pair Plot - ResultLabel - /opt/work/data/src/pairplot_IRIS_REGRESSOR-2-560
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12148_07ed6b645dac47a2.png


---------------------------------------------------------------------------------------
ResultTargetValue Importance Analysis
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12147_5247822cd02242fb.png


---------------------------------------------------------------------------------------


## Get the Job Manifest

Jobs use a manifest to prevent concurrent job and model collisions

#### A manifest contains:

1. A dictionary of model nodes
1. Redis model locations
1. S3 archival locations
1. Tracking data for import and export across environments

In [6]:
job_manifest = get_job_cache_manifest(job_report)
lg(ppj(job_manifest))

{
    "11851": {
        "rloc": "MODELS:_MD_IRIS_REGRESSOR-2-560_26fbf8_0",
        "sloc": "redten-models-west:rt_models_userid_2_job_560_train_453_jobresults_416_modelid_11851_modelkey_MD_IRIS_REGRESSOR-2-560_26fbf8_0.cache.pickle.zlib",
        "target": "SepalLength",
        "tracking_id": "ML_IRIS_REGRESSOR-2-560_0253cc1aba9b4c71b1f69d5d2ae1952",
        "tracking_name": "IRIS_REGRESSOR-2-560",
        "tracking_type": "UseTargetColAndUnits"
    },
    "11852": {
        "rloc": "MODELS:_MD_IRIS_REGRESSOR-2-560_26fbf8_1",
        "sloc": "redten-models-west:rt_models_userid_2_job_560_train_453_jobresults_416_modelid_11852_modelkey_MD_IRIS_REGRESSOR-2-560_26fbf8_1.cache.pickle.zlib",
        "target": "PetalLength",
        "tracking_id": "ML_IRIS_REGRESSOR-2-560_0253cc1aba9b4c71b1f69d5d2ae1952",
        "tracking_name": "IRIS_REGRESSOR-2-560",
        "tracking_type": "UseTargetColAndUnits"
    },
    "11853": {
        "rloc": "MODELS:_MD_IRIS_REGRESSOR-2-560_26fbf8_2",
       

## Multiple Models stored in Redis

Here's how models are stored in the Redis machine learning data store

![Multi-Model storage in Redis](http://i.imgur.com/D5tawCJ.gif)

## Search Jobs in Elasticsearch

Red10 is running an ELK stack for searching by user, dataset identifiers, and jobs.

![ELK](http://i.imgur.com/cUCebca.png)

Once a job completes it is automatically published to Elasticsearch

In [7]:
search_req = {
        "title" : "Hello", # job title with completion
        "dsname" : "", # dataset name with completion
        "desc" : "", # description with completion
        "features" : "", # feature search
        "target_column" : "" # name of target column for this analysis
    }
job_search = {}
job_res = {}
if len(search_req) == 0 :
    boom("Please create a valid search request")
else:
    job_res = search_ml_jobs(search_req)

    if job_res["status"] != "SUCCESS":
        boom("Job=" + str(job_id) + " failed with status=" + str(job_res["status"]) + " err=" + str(job_res["error"]))
    else:
        job_search = job_res["record"]
        anmt("Job Matches=" + str(len(job_search)))
        lg(ppj(job_search), 5)

Searching ML Jobs url=https://redten.io/ml/search/
SUCCESS - Job Search Response Status=200 Reason=OK
Found Job={'target_column': '', 'desc': '', 'dsname': '', 'features': '', 'title': 'Hello'} results
Job Matches=1
{
    "jobs": [
        {
            "algo_name": "xgb-regressor",
            "desc": "This is a Description for - Hello World - 2017-05-26-08-11-22",
            "ds_name": "iris_regressor",
            "feature_column_names": [
                "SepalLength",
                "SepalWidth",
                "PetalLength",
                "PetalWidth",
                "ResultTargetValue"
            ],
            "ignore_features": [
                "ResultLabel"
            ],
            "images": [
                {
                    "id": 12135,
                    "image": "https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_559_12135_6c01fa6f6cba42dd.png",
                    "title": "ResultTargetValue Importance Analysis"
                },
               

## Get the Recent Machine Learning Jobs

In [8]:
debug = False
user_token = user_login(rt_user, rt_pass, rt_url)
auth_headers = {
                "Authorization" : "JWT " + str(user_token)
            }
resource_url = rt_url + "/ml/run/"
query_params = {}
post_data = {}
uni = uni_key()

# Get the ML Job
resource_url = rt_url + "/ml/jobs/"

lg("Running Get ML Job url=" + str(resource_url), 6)
get_response = requests.get(resource_url, params=query_params, data=post_data, headers=auth_headers)

if get_response.status_code != 201 and get_response.status_code != 200:
    lg("Failed with GET Response Status=" + str(get_response.status_code) + " Reason=" + str(get_response.reason), 0)
    lg("Details:\n" + str(get_response.text) + "\n", 0)        
else:
    lg("SUCCESS - GET Response Status=" + str(get_response.status_code) + " Reason=" + str(get_response.reason)[0:10], 5)

    as_json = True
    record = {}
    if as_json:
        record = json.loads(get_response.text)
        lg(ppj(record))
# end of post for running an ML Job

Running Get ML Job url=https://redten.io/ml/jobs/
SUCCESS - GET Response Status=200 Reason=OK
{
    "jobs": [
        {
            "algo_name": "xgb-regressor",
            "control_state": "active",
            "created": "2017-05-26 17-49-23",
            "csv_file": "/opt/work/data/src/iris.csv",
            "desc": "This is a Description for - Hello World - 2017-05-26-17-49-22",
            "ds_name": "iris_regressor",
            "feature_column_names": [
                "SepalLength",
                "SepalWidth",
                "PetalLength",
                "PetalWidth",
                "ResultTargetValue"
            ],
            "id": 560,
            "ignore_features": [
                "ResultLabel"
            ],
            "images": [
                "https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12158_68fe500cd7564213.png",
                "https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12157_0180f59977c44413.png",
                

## Make a new Prediction using the pre-trained XGB Regressor Models

In [9]:
predict_row = {
                "SepalLength"       : 5.4,
                "SepalWidth"        : 3.4,
                "PetalLength"       : 1.7,
                "PetalWidth"        : 0.2,
                "ResultTargetValue" : 0
            }

post_data = {
            "use_cached_job_id" : job_id,
            "predict_this_data" : predict_row,
            "title" : title,
            "desc" : desc,
            "ds_name" : ds_name,
            "feature_column_names" : feature_column_names,
            "ignore_features" : ignore_features,
            "csv_file" : csv_file,
            "rloc" : rloc,
            "sloc" : sloc,
            "units_ahead" : units_ahead,
            "algo_name" : algo_name,
            "target_column_values" : target_column_values,
            "target_column_name" : target_column_name,
            "label_column_name" : label_column_name,
            "prediction_type" : "Predict",
            "ml_type" : "Predict with Filter",
            "user_id" : 2,
            "train" : {
                    "base_score" : 0.5,
                    "col_sample_by_level" : 1.0,
                    "col_sample_by_tree" : 0.8,
                    "gamma" : 0,
                    "learning_rate" : 0.1,
                    "max_delta_step" : 0,
                    "max_depth" : 6,
                    "min_child_weight" : 1,
                    "num_estimators" : 1000,
                    "objective" : "reg:linear",
                    "reg_alpha" : 0.0,
                    "reg_lambda" : 1.0,
                    "scaled_position_weight" : 1.0,
                    "seed" : 27,
                    "sub_sample" : 0.8,
                    "debug" : True
            },
            "max_features" : 10,
            "tracking_type" : "",
            "units_ahead_set" : [],
            "units_ahead_type" : "",
            "version" : 1
    }

#
"""
Login again in case the token expired waiting on the analysis which will show in the logs as:

Failed with Post Response Status=401 Reason=Unauthorized
Details:
{"detail":"Signature has expired."}
"""
user_token = rest_login_as_user(rt_user, rt_pass, rt_url)
post_headers = {
                "Content-type" : "application/json",
                "Authorization" : "JWT " + str(user_token)
            }
resource_url = rt_url + "/ml/run/"

lg("Utilizing Cached Models for new Prediction using ML Job url=" + str(resource_url), 6)
post_response = requests.post(resource_url, params=query_params, data=json.dumps(post_data), headers=post_headers)

if post_response.status_code != 201 and post_response.status_code != 200:
    lg("Failed with Post Response Status=" + str(post_response.status_code) + " Reason=" + str(post_response.reason), 0)
    lg("Details:\n" + str(post_response.text) + "\n", 0)
else:
    lg("SUCCESS - Post Response Status=" + str(post_response.status_code) + " Reason=" + str(post_response.reason), 5)

    record = post_response.json()
    lg(ppj(record))
    job_id = int(record["id"])
# end of post for running an ML Job

Utilizing Cached Models for new Prediction using ML Job url=https://redten.io/ml/run/
SUCCESS - Post Response Status=201 Reason=Created
{
    "algo_name": "xgb-regressor",
    "control_state": "active",
    "created": "2017-05-26 17-49-23",
    "csv_file": "/opt/work/data/src/iris.csv",
    "desc": "This is a Description for - Hello World - 2017-05-26-17-49-22",
    "ds_name": "iris_regressor",
    "feature_column_names": [
        "SepalLength",
        "SepalWidth",
        "PetalLength",
        "PetalWidth",
        "ResultTargetValue"
    ],
    "id": 560,
    "ignore_features": [
        "ResultLabel"
    ],
    "manifest_secret_key": "live_2_2e975a62dd094d5bbdea7bbac0ad33b1bdb540f746f429d995d76acfee6af1",
    "manifest_sloc": "redten-models-west:IRIS_REGRESSOR-2-560_manifest.json",
    "max_features": 10,
    "ml_type": "Predict with Filter",
    "prediction_type": "predict",
    "rloc": "",
    "sloc": "",
    "status": "completed",
    "target_column_name": "ResultLabel",
    

## Wait for the Job to finish

In [10]:
job_data = {}
job_report = {}

job_res = {}
if job_id == None:
    boom("Failed to start a new job")
else:
    job_res = wait_on_job(job_id)

    if job_res["status"] != "SUCCESS":
        boom("Job=" + str(job_id) + " failed with status=" + str(job_res["status"]) + " err=" + str(job_res["error"]))
    else:
        job_data = job_res["record"]
        anmt("Job Report:")
        lg(ppj(job_data), 5)
# end of waiting

Waiting on job=560 url=https://redten.io/ml/560/
Job=560 is training - Step 3/10
Job=560 is plotting - Step 7/10
Job=560 is uploading - Step 9/10
Job=560 completed
Job Report:
{
    "job": {
        "algo_name": "xgb-regressor",
        "control_state": "active",
        "created": "2017-05-26 17-49-23",
        "csv_file": "/opt/work/data/src/iris.csv",
        "desc": "This is a Description for - Hello World - 2017-05-26-17-49-22",
        "ds_name": "iris_regressor",
        "feature_column_names": [
            "SepalLength",
            "SepalWidth",
            "PetalLength",
            "PetalWidth",
            "ResultTargetValue"
        ],
        "id": 560,
        "ignore_features": [
            "ResultLabel"
        ],
        "images": [
            {
                "author_name": null,
                "desc": null,
                "id": 12170,
                "image": "https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12170_e4c5d75535004a21.png",
        

## Get the new Prediction results

In [11]:
job_report = {}
if job_id == None:
    boom("Failed to start a new prediction job")
else:
    # show the plots by default
    job_report = get_job_analysis(job_id)

    if len(job_report) == 0:
        boom("Job=" + str(job_id) + " failed making new prediction")
    # end of checking the report had data
# end of get job analysis

Getting analysis for job=560 url=https://redten.io/ml/analysis/560/
SUCCESS - GET Analysis Response Status=200 Reason=OK
Found Job=560 analysis
Joint Plot - ResultTargetValue vs SepalLength - /opt/work/data/src/jointplot_IRIS_REGRESSOR-2-560_4
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12170_e4c5d75535004a21.png


---------------------------------------------------------------------------------------
Joint Plot - PetalWidth vs SepalLength - /opt/work/data/src/jointplot_IRIS_REGRESSOR-2-560_3
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12169_e2025b07c7e24a83.png


---------------------------------------------------------------------------------------
Joint Plot - PetalLength vs SepalLength - /opt/work/data/src/jointplot_IRIS_REGRESSOR-2-560_2
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12168_ff223da922584537.png


---------------------------------------------------------------------------------------
Joint Plot - SepalWidth vs SepalLength - /opt/work/data/src/jointplot_IRIS_REGRESSOR-2-560_1
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12167_9b9870cab7a04c99.png


---------------------------------------------------------------------------------------
Joint Plot - SepalLength vs SepalWidth - /opt/work/data/src/jointplot_IRIS_REGRESSOR-2-560_0
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12166_d9dd29fe5eaa48d3.png


...

---------------------------------------------------------------------------------------
Scatter - SepalWidth vs SepalLength - /opt/work/data/src/scatter_IRIS_REGRESSOR-2-560_1
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12150_8166153a2f87499d.png


---------------------------------------------------------------------------------------
Scatter - SepalLength vs SepalWidth - /opt/work/data/src/scatter_IRIS_REGRESSOR-2-560_0
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12149_bd798d8ff84f46f6.png


---------------------------------------------------------------------------------------
Pair Plot - ResultLabel - /opt/work/data/src/pairplot_IRIS_REGRESSOR-2-560
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12148_07ed6b645dac47a2.png


---------------------------------------------------------------------------------------
ResultTargetValue Importance Analysis
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_560_12147_5247822cd02242fb.png


---------------------------------------------------------------------------------------
